In [ ]:
!pip install pytorch-lightning -q
# Code for TPU packages install
# !curl -q https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev

In [ ]:
import torch
from torchmetrics import AUROC
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import pytorch_lightning as pl
import datetime
import glob
import re
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.datasets import make_multilabel_classification
from sklearn.utils import class_weight
from sklearn.utils.multiclass import unique_labels
from sklearn.utils import resample

import seaborn as sns
import matplotlib.pyplot as plt
from transformers import BertModel, BertTokenizer
import spacy

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

path = '/content/gdrive/MyDrive/b5/'

Mounted at /content/gdrive


In [ ]:
USE_TPU = False
if USE_TPU:
  import torch_xla
  import torch_xla.core.xla_model as xm
  device = xm.xla_device()
else:
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
print(model.config.hidden_size)

# print("Обновляемые параметры модели:")
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print(name)

# for param in model.bert.parameters():
#     param.requires_grad = False

# print("Обновляемые параметры модели:")
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print(name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


768


In [ ]:
# ds_full = pd.read_csv(f'{path}Big-Five_Backstage_extended.csv')
# [print(ds_full.columns[k],"\t ---",np.array(ds_full[:1]).squeeze()[k]) for k in range(len(np.array(ds_full[:1]).squeeze()))]


In [ ]:
# ds_full = pd.read_csv(f'{path}Big-Five_Backstage.csv')

# data = ds_full.iloc[:, [4, -5, -4, -3, -2, -1]].values
# columns = ds_full.columns[[4, -5, -4, -3, -2, -1]]

# df = pd.DataFrame(data, columns=columns)
# print(df)

# texts = df['text'].values
# labels = df[['Extraversion', 'Agreeableness', 'Openness', 'Neuroticism', 'Conscientiousness']].values.astype(float)
# splited_texts = []
# splited_labels = []
# for idx, text in enumerate(texts):
#   s = re.split(r'(?<=[.!?])\s+', text)
#   splited_texts.extend(s)
#   splited_labels.extend([labels[idx] for ss in range(len(s))])

# y_train = np.array(splited_labels)
# X_train = np.array(splited_texts)

# label_counts = np.sum(y_train, axis=0)
# print("Original label counts:", label_counts)

# # Подсчет весов классов для учета дисбаланса
# class_weights = class_weight.compute_class_weight('balanced', classes=unique_labels(y_train), y=np.argmax(y_train, axis=1))
# print("Class weights:", class_weights)

# resampled_dataset = ResampledDataset(X_train, y_train, class_weights, random_state=42)

# print(resampled_dataset)

# X_resampled = []
# y_resampled = []
# for class_idx in range(y_train.shape[1]):
#     # Определение числа образцов для текущего класса
#     num_samples = int(class_weights[class_idx] * label_counts[class_idx])
#     # Пересэмплирование текущего класса
#     X_class = X_train[y_train[:, class_idx] == 1]
#     y_class = y_train[y_train[:, class_idx] == 1]
#     X_resampled_class, y_resampled_class = resample(X_class, y_class, n_samples=num_samples, replace=True, random_state=42)
#     X_resampled.extend(X_resampled_class)
#     y_resampled.extend(y_resampled_class)

# # Преобразование списков в массивы
# X_resampled = np.array(X_resampled)
# y_resampled = np.array(y_resampled)

# # Оценка нового распределения меток
# resampled_label_counts = np.sum(y_resampled, axis=0)

# splited_texts = X_resampled
# splited_labels = y_resampled

In [ ]:
keys =  ['Extraversion', 'Neuroticism', 'Agreeableness', 'Conscientiousness', 'Openness']
df1 = pd.read_csv(f'{path}Big-Five_Backstage.csv')
df = pd.read_csv(f'{path}essays/essays.csv', encoding='cp1252', delimiter=',', quotechar='"')
df = df.rename(columns={'TEXT': 'text',
                        'cEXT': 'Extraversion',
                        'cNEU': 'Neuroticism',
                        'cAGR': 'Agreeableness',
                        'cCON': 'Conscientiousness',
                        'cOPN': 'Openness'})
keys =  ['Extraversion', 'Neuroticism', 'Agreeableness', 'Conscientiousness', 'Openness']
df[keys] = df[keys].replace({'y': 1.0, 'n': 0.0})
df

,#AUTHID,text,Extraversion,Neuroticism,Agreeableness,Conscientiousness,Openness
0,1997_504851.txt,"Well, right now I just woke up from a mid-day ...",0.0,1.0,1.0,0.0,1.0
1,1997_605191.txt,"Well, here we go with the stream of consciousn...",0.0,0.0,1.0,0.0,0.0
2,1997_687252.txt,An open keyboard and buttons to push. The thin...,0.0,1.0,0.0,1.0,1.0
3,1997_568848.txt,I can't believe it! It's really happening! M...,1.0,0.0,1.0,1.0,0.0
4,1997_688160.txt,"Well, here I go with the good old stream of co...",1.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...
2463,2004_493.txt,I'm home. wanted to go to bed but remembe...,0.0,1.0,0.0,1.0,0.0
2464,2004_494.txt,Stream of consiousnesssskdj. How do you s...,1.0,1.0,0.0,0.0,1.0
2465,2004_497.txt,"It is Wednesday, December 8th and a lot has be...",0.0,0.0,1.0,0.0,0.0
2466,2004_498.txt,"Man this week has been hellish. Anyways, now i...",0.0,1.0,0.0,0.0,1.0


In [ ]:
texts = pd.concat([df['text'], df1['text']], ignore_index=True)
labels = pd.concat([df[keys], df1[keys]], ignore_index=True).T[:1].T

labels

,Extraversion
0,0.0
1,0.0
2,0.0
3,1.0
4,1.0
...,...
5721,0.0
5722,0.0
5723,0.0
5724,0.0


In [ ]:
indices_to_labels = { 0:'e', 1:'n', 2:'a', 3:'c', 4:'o' }
labels_to_indices = { 'e':0, 'n':1, 'a':2, 'c':3, 'o':4 }

TYPE_TO_PROCESS = 'e'
cur_idx = labels_to_indices[TYPE_TO_PROCESS]

In [ ]:
label_tensor = torch.tensor(np.array(labels), device=device)
label_tensor

tensor([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]], device='cuda:0', dtype=torch.float64)

In [ ]:
undersampler = RandomUnderSampler(random_state=42)
indices, under_label_tensor = undersampler.fit_resample(np.array([range(len(label_tensor))]).T, label_tensor.cpu())

In [ ]:
# train_texts, test_texts, train_labels_raw, test_labels_raw = train_test_split(splited_texts, splited_labels, test_size=0.2, random_state=42)
train_texts, test_texts, train_labels_raw, test_labels_raw = train_test_split(texts, labels, test_size=0.2, random_state=42)
use_predefined_encodings = True

if not use_predefined_encodings:
  encodings = tokenizer(list(texts), truncation=True, return_tensors="pt", max_length=200, padding='max_length').to(device)

  torch.save(encodings, f"{path}dict/bert/encodings.pt")
else:
  encodings = torch.load(f"{path}dict/bert/encodings.pt").to(device)

enc = {}
enc['input_ids'] = encodings['input_ids'][indices].to(device)
enc['attention_mask'] = encodings['attention_mask'][indices].to(device)
enc['token_type_ids'] = encodings['token_type_ids'][indices].to(device)

train_encodings = {}
test_encodings = {}
train_encodings['input_ids'], test_encodings['input_ids'], train_encodings['attention_mask'],  test_encodings['attention_mask'], train_encodings['token_type_ids'], test_encodings['token_type_ids'], train_labels, test_labels = train_test_split(enc['input_ids'], enc['attention_mask'], enc['token_type_ids'], torch.tensor(np.array([under_label_tensor]), dtype=torch.float, device=device).T, test_size=0.2, random_state=42, stratify = under_label_tensor)

train_dataset = torch.utils.data.TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_encodings['token_type_ids'], train_labels)

test_dataset = torch.utils.data.TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_encodings['token_type_ids'], test_labels)

KeyError: 'Invalid key. Only three types of key are available: (1) string, (2) integers for backend Encoding, and (3) slices for data subsetting.'

In [ ]:
torch.tensor(np.array([under_label_tensor])).size()

torch.Size([1, 4746])

In [ ]:
# use_predefined_encodings = False

# if not use_predefined_encodings:
#   train_dataset = CustomDataset(train_texts, train_labels, tokenizer, 200)
#   test_dataset = CustomDataset(test_texts, test_labels, tokenizer, 200)

#   torch.save(train_dataset, f"{path}train_dataset.pth")
#   torch.save(test_dataset, f"{path}test_dataset.pth")
# else:
#   train_dataset = torch.load(f"{path}train_dataset.pth")
#   test_dataset = torch.load(f"{path}test_dataset.pth")

# train_dataset[0]

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=False)

In [ ]:
class MyLightningModule(pl.LightningModule):
    def __init__(self, bert):
        super().__init__()
        self.loss = torch.nn.BCELoss()
        self.bert = bert.to(device)
        self.d = torch.nn.Dropout(0.2)
        self.l1 = torch.nn.Linear(self.bert.config.hidden_size, 128)
        self.l2 = torch.nn.Linear(128, 256)
        self.l3 = torch.nn.Linear(256, 128)
        self.l4 = torch.nn.Linear(128, 1)
        self.s = torch.nn.Sigmoid()

    def forward(self, input_ids, attention_mask, token_type_ids):
        _, output = self.bert(input_ids=input_ids.squeeze(), attention_mask=attention_mask.squeeze(), token_type_ids=token_type_ids.squeeze(), return_dict=False)
        output = self.d(output)
        output = self.l1(output)
        output = torch.relu(output)
        output = self.l2(output)
        output = torch.relu(output)
        output = self.l3(output)
        output = torch.relu(output)
        output = self.l4(output)
        output = self.s(output)
        return output

    def training_step(self, batch, batch_idx):
        input_ids, attention_mask, token_type_ids, labels = batch
        outputs = self(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        loss = self.loss(outputs, labels.float())

        self.log('train_loss', loss, on_step=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids, attention_mask, token_type_ids, labels = batch
        outputs = self(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        loss = self.loss(outputs, labels.float())

        self.log('val_loss', loss, on_step=True)

    def configure_optimizers(self):
        return torch.optim.AdamW(filter(lambda p: p.requires_grad, self.parameters()), lr=1e-5)

In [ ]:
def get_best_checkpoint_path(checkpoint_dir):
    checkpoint_files = glob.glob(f"{checkpoint_dir}/*.ckpt")

    if not checkpoint_files:
        return None

    best_checkpoint_path = None
    best_loss = float('inf')

    for checkpoint_file in checkpoint_files:
        file_parts = checkpoint_file.split('-')
        val_loss = float(file_parts[-1][:-5].split('=')[1])

        if val_loss < best_loss:
            best_loss = val_loss
            best_checkpoint_path = checkpoint_file

    return best_checkpoint_path

In [ ]:
lModel = MyLightningModule(model)

current_date = datetime.datetime.now().strftime("%Y-%m-%d")

num_epochs = 20
trainer = pl.Trainer(
    limit_test_batches=0.5,
    max_epochs=num_epochs,
    log_every_n_steps=100,
    logger=pl.loggers.TensorBoardLogger(f'{path}logs/', name=f'bert-b5-{current_date}'),
    callbacks=[pl.callbacks.ModelCheckpoint(dirpath=f'{path}checkpoints/', filename=f'bert-b5-{current_date}-{{epoch}}-{{val_loss:.3f}}', monitor='val_loss', mode='min', save_top_k=1, verbose=True, enable_version_counter=True)]
)

best_checkpoint_path = None # get_best_checkpoint_path(f'{path}checkpoints')
trainer.fit(lModel, ckpt_path=best_checkpoint_path, train_dataloaders=train_loader, val_dataloaders=test_loader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type      | Params
------------------------------------
0 | loss  | BCELoss   | 0     
1 | model | BertModel | 109 M 
2 | d     | Dropout   | 0     
3 | l1    | Linear    | 98.4 K
4 | l2    | Linear    | 33.0 K
5 | l3    | Linear    | 32.9 K
6 | l4    | Linear    | 129   
7 | s     | Sigmoid   | 0     
------------------------------------
109 M     Trainable params
0         Non-trainable params
109 M     Total params
438.587   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 475: 'val_loss' reached 0.68071 (best 0.68071), saving model to '/content/gdrive/.shortcut-targets-by-id/1-FlGYYbCrLYtFwF16K3n-LPIeS1wfsMY/b5/checkpoints/bert-b5-2024-05-22-epoch=0-val_loss=0.681.ckpt' as top 1


...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 9, global step 4750: 'val_loss' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 10, global step 5225: 'val_loss' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 11, global step 5700: 'val_loss' was not in top 1


In [ ]:
trainer.test(dataloaders=test_loader)

In [ ]:
def to_cpu(inputs):
  t = inputs.detach().cpu() if inputs.is_cuda else inputs.detach()
  return t.numpy().astype(float)

In [ ]:
lModel.eval()
lModel.to(device)
result_values = []
res_lab_values = []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, token_type_ids, labels = batch
        outputs = lModel(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        for idx in range(len(outputs)):
          label_names = ['Extroversion', 'Neuroticism', 'Agreeableness', 'Conscientiousness', 'Openness']
          # result = {label_names[i]: (predictions[idx] > 0.5)[i] for i in range(len(label_names[:1]))}
          # res_lab = {label_names[i]: (labels[idx]> 0.5)[i] for i in range(len(label_names[:1]))}

          result = {label_names[i]: (to_cpu(outputs[idx])> 0.5)[i].astype(float) for i in range(len(label_names[:1]))}
          res_lab = {label_names[i]: (to_cpu(labels[idx])> 0.5)[i].astype(float) for i in range(len(label_names[:1]))}
          result_values.append(np.array(list(result.values())))
          res_lab_values.append(np.array(list(res_lab.values())))
          # print(res_lab)
          print(result)
          print()


NameError: name 'to_cpu' is not defined

In [ ]:
input_ = "Hi? what is your name?"
lModel.eval()
lModel.to(device)
with torch.no_grad():
    enc = tokenizer(input_, truncation=True, return_tensors="pt", max_length=200, padding='max_length').to(device)
    outputs = lModel(input_ids=enc['input_ids'], attention_mask=enc['attention_mask'], token_type_ids=enc['token_type_ids'])
    print(outputs)
    raw_predicted_labels = outputs.float().tolist()[0]
    predicted_labels = (outputs > 0.5).float().tolist()[0]
    print(predicted_labels)
    label_names = ['Extroversion', 'Neuroticism', 'Agreeableness', 'Conscientiousness', 'Openness']
    result1 = {label_names[i]: predicted_labels[i] for i in range(len(label_names[:1]))}
    result2 = {label_names[i]: raw_predicted_labels[i] for i in range(len(label_names[:1]))}
    print(result1)
    print(result2)


ValueError: not enough values to unpack (expected 2, got 1)